In [2]:
# Training with new ntuple

# prepare

## 1 - Load modules

In [1]:
import sys
sys.path.append("..") # add self-defined module in the parent path
sys.path.append("../..") # add self-defined module in the parent path
import time

from array import array
import datetime
import keras.backend
from keras.models import Sequential, Model, load_model
from keras.layers import Concatenate, Dense, Input
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adagrad, SGD, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from sklearn import preprocessing
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from run.get_arrays import *
from lfv_pdnn_code_v1.train import model
from lfv_pdnn_code_v1.train.train_utils import *
from lfv_pdnn_code_v1.common.common_utils import *

Using TensorFlow backend.


In [2]:
# Constants
new_bkg_path = "E:/data/new_ntuple/mc16a"
old_bkg_path = "E:/data/lfv/ntuples_last_run/TestData/data_npy"  # Windows can recognize both "/" and "\"

## 2 - Load new array
### a) load background samples

In [3]:
# Set parent path to access data
xb_dict_new = get_new_bkg(new_bkg_path)

Loading new background array.
xb_di_boson shape: (69860, 24)
xb_top_quark shape: (68355, 24)
xb_w_jets shape: (2575, 24)
xb_z_ll shape: (407, 24)
New background organized with dict: xb_dict_new
Adding all background together.
xb shape: (141197, 24)


### b) load signal samples
when all up different signal mass point together, two method applied:
* xs: directly add each mass point signal together
* xs_norm: each mass point sample is normalized before add together/


In [3]:
# Initialize
mass_min = 5000
mass_max = 0
xs_studied = np.array([])
mass_scan_map = [500, 2000]
#mass_scan_map = [500]
xs_dict_new = {}

# Load
print "Loading new signal array."
print "\nOrganizing new signal with dict: xs_dict_new."
data_path = "/mnt/e/data/new_ntuple/mc16a"
xs = np.array([])
xs_norm = np.array([])
for i, mass in enumerate(mass_scan_map):
    # load signal
    xs_add = np.load(data_path + "/signal/rpv_emu_{}GeV.npy".format(mass))
    xs_temp = np.load(data_path + "/signal/rpv_etau_{}GeV.npy".format(mass))
    xs_add = np.concatenate((xs_add, xs_temp))
    xs_temp = np.load(data_path + "/signal/rpv_mutau_{}GeV.npy".format(mass))
    xs_add = np.concatenate((xs_add, xs_temp))
    # add to dict xs_dict_new
    print "adding {}GeV signal to xs_dict_new".format(mass), xs_add.shape
    xs_dict_new['{}GeV'.format(mass)] = xs_add
    # add to full signals
    if len(xs) == 0:
        xs = xs_add.copy()
        xs_add_norm = modify_array(xs_add, weight_id = -1, norm = True)
        xs_norm = xs_add_norm.copy()
    else:
        xs = np.concatenate((xs, xs_add))
        xs_add_norm = modify_array(xs_add, weight_id = -1, norm = True)
        xs_norm = np.concatenate((xs_norm, xs_add_norm))
print "adding all signal to xs_dict_new"
xs_dict_new['all'] = xs
print "adding all_norm signal to xs_dict_new"
xs_dict_new['all_norm'] = xs_norm

print "\nDone."

## 2 - Load old array
### a) load old background samples

In [4]:
# Load
print "Loading new background array."
xb_di_boson_old = np.load('/mnt/e/data/lfv/ntuples_last_run/TestData/data_npy/tree_bkg1.npy')
xb_drell_yan_old = np.load('/mnt/e/data/lfv/ntuples_last_run/TestData/data_npy/tree_bkg2.npy')
xb_top_quark_old = np.load('/mnt/e/data/lfv/ntuples_last_run/TestData/data_npy/tree_bkg3.npy')
xb_w_jets_old = np.load('/mnt/e/data/lfv/ntuples_last_run/TestData/data_npy/tree_bkg4.npy')
xb_z_ll_old = np.load('/mnt/e/data/lfv/ntuples_last_run/TestData/data_npy/tree_bkg5.npy')
xb_old = np.concatenate((xb_di_boson_old, xb_drell_yan_old, xb_top_quark_old, xb_w_jets_old, xb_z_ll_old))

# Organize with dict
print "\nOrganizing new background with dict: xb_dict_old."
xb_dict_old = {}
xb_dict_old['di_boson'] = xb_di_boson_old
xb_dict_old['drell_yan'] = xb_drell_yan_old
xb_dict_old['top_quark'] = xb_top_quark_old
xb_dict_old['w_jets'] = xb_w_jets_old
xb_dict_old['z_ll'] = xb_z_ll_old
xb_dict_old['all'] = xb_old

print "\nDone."

### b) load old signal samples

In [5]:
# Initialize
mass_min = 5000
mass_max = 0
xs_old = np.array([])
xs_old_norm = np.array([])
xs_old_max_2400 = np.array([])
xs_old_max_2400_norm = np.array([])

xs_dict_old = {}
mass_scan_map = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22,24,26,28,30,35,40,45,50]

# Load
print "Loading old signal array."
print "\nOrganizing old signal with dict: xs_dict_old."
for i, mass in enumerate(mass_scan_map):
    # load signal
    xs_add = np.load('/mnt/e/data/lfv/ntuples_last_run/train_array_0909/data_npy/emu/rpv_{}00GeV.npy'.format(mass))
    xs_temp = np.load('/mnt/e/data/lfv/ntuples_last_run/train_array_0909/data_npy/etau/rpv_{}00GeV.npy'.format(mass))
    xs_add = np.concatenate((xs_add, xs_temp))
    xs_temp = np.load('/mnt/e/data/lfv/ntuples_last_run/train_array_0909/data_npy/mutau/rpv_{}00GeV.npy'.format(mass))
    xs_add = np.concatenate((xs_add, xs_temp))
    # add to dict xs_dict_new
    #print "adding {}00GeV signal to xs_dict_old".format(mass)
    xs_dict_old['{}00GeV'.format(mass)] = xs_add
    # add to full signals
    if len(xs_old) == 0:
        xs_old = xs_add.copy()
        xs_add_norm = modify_array(xs_add, weight_id = -1, norm = True)
        xs_old_norm = xs_add_norm.copy()
    else:
        xs_old = np.concatenate((xs_old, xs_add))
        xs_add_norm = modify_array(xs_add, weight_id = -1, norm = True)
        xs_old_norm = np.concatenate((xs_old_norm, xs_add_norm))
    if mass <= 24:
        xs_old_max_2400 = xs_old.copy()
        xs_old_max_2400_norm = xs_old_norm.copy()
        
xs_dict_old['all'] = xs_old
xs_dict_old['all_norm'] = xs_old_norm
xs_dict_old['max_2400'] = xs_old_max_2400
xs_dict_old['max_2400_norm'] = xs_old_max_2400_norm

# load data with only mass 500, 1000, 2000
xs_old_3point = np.array([])
xs_old_3point_norm = np.array([])
for i, mass in enumerate([5, 10, 20]):
    # load signal
    xs_add = np.load('/mnt/e/data/lfv/ntuples_last_run/train_array_0909/data_npy/emu/rpv_{}00GeV.npy'.format(mass))
    xs_temp = np.load('/mnt/e/data/lfv/ntuples_last_run/train_array_0909/data_npy/etau/rpv_{}00GeV.npy'.format(mass))
    xs_add = np.concatenate((xs_add, xs_temp))
    xs_temp = np.load('/mnt/e/data/lfv/ntuples_last_run/train_array_0909/data_npy/mutau/rpv_{}00GeV.npy'.format(mass))
    xs_add = np.concatenate((xs_add, xs_temp))
    # add to dict xs_dict_new
    #print "adding {}00GeV signal to xs_dict_old".format(mass)
    xs_dict_old['{}00GeV'.format(mass)] = xs_add
    # add to full signals
    if len(xs_old_3point) == 0:
        xs_old_3point = xs_add.copy()
        xs_add_norm = modify_array(xs_add, weight_id = -1, norm = True)
        xs_old_3point_norm = xs_add_norm.copy()
    else:
        xs_old_3point = np.concatenate((xs_old_3point, xs_add))
        xs_add_norm = modify_array(xs_add, weight_id = -1, norm = True)
        xs_old_3point_norm = np.concatenate((xs_old_3point_norm, xs_add_norm))
xs_dict_old['3_point'] = xs_old_3point
xs_dict_old['3_point_norm'] = xs_old_3point_norm

# load data with only mass 500, 700, 1000, 2000GeV
xs_old_4point = np.array([])
xs_old_4point_norm = np.array([])
for i, mass in enumerate([5, 7, 10, 20]):
    # load signal
    xs_add = np.load('/mnt/e/data/lfv/ntuples_last_run/train_array_0909/data_npy/emu/rpv_{}00GeV.npy'.format(mass))
    xs_temp = np.load('/mnt/e/data/lfv/ntuples_last_run/train_array_0909/data_npy/etau/rpv_{}00GeV.npy'.format(mass))
    xs_add = np.concatenate((xs_add, xs_temp))
    xs_temp = np.load('/mnt/e/data/lfv/ntuples_last_run/train_array_0909/data_npy/mutau/rpv_{}00GeV.npy'.format(mass))
    xs_add = np.concatenate((xs_add, xs_temp))
    # add to dict xs_dict_new
    #print "adding {}00GeV signal to xs_dict_old".format(mass)
    xs_dict_old['{}00GeV'.format(mass)] = xs_add
    # add to full signals
    if len(xs_old_4point) == 0:
        xs_old_4point = xs_add.copy()
        xs_add_norm = modify_array(xs_add, weight_id = -1, norm = True)
        xs_old_4point_norm = xs_add_norm.copy()
    else:
        xs_old_4point = np.concatenate((xs_old_4point, xs_add))
        xs_add_norm = modify_array(xs_add, weight_id = -1, norm = True)
        xs_old_4point_norm = np.concatenate((xs_old_4point_norm, xs_add_norm))
xs_dict_old['4_point'] = xs_old_4point
xs_dict_old['4_point_norm'] = xs_old_4point_norm

print "\nDone."

# Make plots

## 1 - kinematic plots for emu channel

In [6]:
xs_dict_new['500GeV'][:,-1]

In [14]:
sampel_key = 'all'
#plt.hist(xs_dict_old[sampel_key][:,0], bins=50, weights=xs_dict_old[sampel_key][:,-1], histtype='step', label='signal', range=(0, 10000), density = False) # blue
plt.hist(xs_dict_new[sampel_key][:,0], bins=50, weights=xs_dict_new[sampel_key][:,-1], histtype='step', label='signal', range=(0, 10000), density = False) # orange
plt.show()

# bkg
sampel_key = 'all'
#plt.hist(xb_dict_old[sampel_key][:,0], bins=50, weights=xb_dict_old[sampel_key][:,-1], histtype='step', label='signal', range=(0, 10000), density = True)
plt.hist(xb_dict_new[sampel_key][:,0], bins=50, weights=xb_dict_new[sampel_key][:,-1], histtype='step', label='signal', range=(0, 10000), density = True)
plt.show()

sampel_key = 'di_boson'
plt.hist(xb_dict_old[sampel_key][:,0], bins=50, weights=xb_dict_old[sampel_key][:,-1], histtype='step', label='signal', range=(500, 5000), density = True)
sampel_key = 'drell_yan'
plt.hist(xb_dict_old[sampel_key][:,0], bins=50, weights=xb_dict_old[sampel_key][:,-1], histtype='step', label='signal', range=(500, 5000), density = True)
sampel_key = 'top_quark'
plt.hist(xb_dict_old[sampel_key][:,0], bins=50, weights=xb_dict_old[sampel_key][:,-1], histtype='step', label='signal', range=(500, 5000), density = True)
sampel_key = 'w_jets'
plt.hist(xb_dict_old[sampel_key][:,0], bins=50, weights=xb_dict_old[sampel_key][:,-1], histtype='step', label='signal', range=(500, 5000), density = True)
sampel_key = 'z_ll'
plt.hist(xb_dict_old[sampel_key][:,0], bins=50, weights=xb_dict_old[sampel_key][:,-1], histtype='step', label='signal', range=(500, 5000), density = True)
plt.show()

In [6]:
xs_emu = modify_array(xs_dict_new['all'], weight_id = -1, select_channel = True, channel_id = -4)
xb_emu = modify_array(xb_dict_new['all'], weight_id = -1, select_channel = True, channel_id = -4)
xs_etau = modify_array(xs_dict_new['all'], weight_id = -1, select_channel = True, channel_id = -3)
xb_etau = modify_array(xb_dict_new['all'], weight_id = -1, select_channel = True, channel_id = -3)
xs_mutau = modify_array(xs_dict_new['all'], weight_id = -1, select_channel = True, channel_id = -2)
xb_mutau = modify_array(xb_dict_new['all'], weight_id = -1, select_channel = True, channel_id = -2)

plt.hist(xs_etau[:,0], bins=50, weights=xs_etau[:,-1], histtype='step', label='etau', range=(0, 10000), density = False)
plt.hist(xs_mutau[:,0], bins=50, weights=xs_mutau[:,-1], histtype='step', label='etau', range=(0, 10000), density = False)
plt.legend(prop={'size': 10})
plt.xlabel("mass GeV")
plt.ylabel("events")
plt.show()

plt.hist(xb_emu[:,0], bins=50, weights=xb_emu[:,-1], histtype='step', label='signal', range=(1500, 2500), density = True)
plt.show()

print xb_dict_new['all'].shape
print xb_dict_old['all'].shape

In [7]:
xs_emu = modify_array(xs_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4)
xb_emu = modify_array(xb_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4)
xs_etau = modify_array(xs_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -3)
xb_etau = modify_array(xb_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -3)
xs_mutau = modify_array(xs_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -2)
xb_mutau = modify_array(xb_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -2)

plt.hist(xs_etau[:,0], bins=50, weights=xs_etau[:,-1], histtype='step', label='etau', range=(0, 10000), density = False)
plt.hist(xs_mutau[:,0], bins=50, weights=xs_mutau[:,-1], histtype='step', label='etau', range=(0, 10000), density = False)
plt.legend(prop={'size': 10})
plt.xlabel("mass GeV")
plt.ylabel("events")
plt.show()

plt.hist(xb_emu[:,0], bins=50, weights=xb_emu[:,-1], histtype='step', label='signal', range=(1500, 2500), density = True)
plt.show()

In [78]:
xs_emu = modify_array(xb_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -2,
                      norm = True, shuffle = True, shuffle_seed = int(time.time()))
xb_emu = modify_array(xb_dict_new['all'], weight_id = -1, remove_negative_weight = True, select_channel = True, channel_id = -2,
                      norm = True, shuffle = True)
#"""
MakePlots(xs_emu, xb_emu, 0, bins = 50, range = (1800, 2200)  , density = False,
          xlabel = "di-Lepton mass / GeV", ylabel = "events", show_plot = True)
"""
MakePlots(xs_emu, xb_emu, 1, bins = 50, range = (0, 1000)  , density = True,
          xlabel="Ele_pt / GeV"          , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 2, bins = 50, range = (-3, 3)    , density = True,
          xlabel="Ele_eta"               , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 3, bins = 50, range = (-3.2, 3.2), density = True,
          xlabel="Ele_phi"               , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 5, bins = 50, range = (0, 5000)  , density = True,
          xlabel="Mu_pt / GeV"           , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 6, bins = 50, range = (-3, 3)    , density = True,
          xlabel="Mu_eta"                , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 7, bins = 50, range = (-3.2, 3.2), density = True,
          xlabel="Mu_phi"                , ylabel="Events"  , show_plot = True)

MakePlots(xs_emu, xb_emu, 15, bins = 50, range = (0, 1000)  , density = True,
          xlabel="di-Lepton_pt"          , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 16, bins = 50, range = (-3, 3)    , density = True,
          xlabel="di-Lepton_eta"         , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 17, bins = 50, range = (-3.2, 3.2), density = True,
          xlabel="di-Lepton_phi"         , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 18, bins = 50, range = (-3.2, 3.2), density = True,
          xlabel="di-Lepton_Dphi"        , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 18, bins = 50, range = (0, 5)     , density = True,
          xlabel="di-Lepton_DR"          , ylabel="Events"  , show_plot = True)
"""

# Training Test

## 1 - Train with individual mass point (new vs old)

In [21]:
selected_features_emu = [0, 1, 2, 3, 5, 6, 7, 15, 16, 17, 18, 19]
selected_features_etau = [0, 1, 2, 3, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19]
selected_features_mutau = [0, 5, 6, 7, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19]

def train_single_mass(sig_dict, bkg_dict, channel_name, channel_id, mass_name, model_name, epochs=20,
                      selected_features = [0, 1, 2, 3, 5, 6, 7, 15, 16, 17, 18, 19], select_bkg_mass = True,
                      self_defined_mass = False, minm = None, maxm = None):
    print "Training {}.".format(channel_name)
    # get data
    print "Loading signal."
    xs_emu = modify_array(sig_dict[mass_name], weight_id = -1, select_channel = True, channel_id = channel_id,
                          norm = True, shuffle = True, shuffle_seed = int(time.time()))
    mass_min, mass_max = get_mass_range(xs_emu[:, 0], xs_emu[:, -1])
    num = 0
    total = 0
    for x in xs_emu[:, 0]:
        #if x > 15000:
        #    print "huge mass:", x
        total += x
        num += 1.0
    print total / num
    print "mass average:", np.average(xs_emu[:, 0], weights=xs_emu[:, -1])
    if self_defined_mass:
        mass_min = minm
        mass_max = maxm
    if select_bkg_mass:
        print "Loading background with mass range:", mass_min, "to", mass_max
    xb_emu = modify_array(bkg_dict['all'], weight_id = -1, remove_negative_weight = True, select_channel = True, channel_id = channel_id,
                          select_mass = select_bkg_mass, mass_id = 0, mass_min = mass_min, mass_max = mass_max,
                          reset_mass = True, reset_mass_array = xs_emu, reset_mass_id = 0,
                          norm = True, shuffle = True, shuffle_seed = int(time.time()))
    # set model and train
    model_deep = model.model_0913(model_name, len(selected_features))
    model_deep.compile()
    model_deep.prepare_array(xs_emu, xb_emu, selected_features)
    model_deep.train(epochs = epochs, val_split = 0.1, verbose = 0)
    # save model
    model_deep.save_model("/mnt/e/data/new_ntuple/model/" + model_name + ".h5")
    # performance plots
    model_deep.show_performance()
    print "\nDone."

### A) mass = 500 GeV

#### a) old ntuple
[emu]

In [24]:
#"""
from lfv_pdnn_code_v1.train import model, train_utils
from lfv_pdnn_code_v1.train.train_utils import *

reload(model)
reload(print_helper)
reload(train_utils)
from lfv_pdnn_code_v1.train.train_utils import *
#"""
model = train_single_mass(xs_dict_old, xb_dict_old, 'emu', -4, '500GeV', "model_mass_500_emu", epochs=5,
                          selected_features = selected_features_emu)


*************************
[etau]

In [12]:
train_single_mass(xs_dict_old, xb_dict_old, 'etau', -3, '500GeV', "model_mass_500_etau", epochs=30, selected_features = selected_features_etau)

**********
[mutau]

In [16]:
train_single_mass(xs_dict_old, xb_dict_old, 'mutau', -2, '500GeV', "model_mass_500_mutau", epochs=30, selected_features = selected_features_mutau)

#### b) new ntuple
[emu]

In [13]:
train_single_mass(xs_dict_new, xb_dict_new, 'emu', -4, '500GeV', "model_mass_500_emu_new", epochs=30, selected_features = selected_features_emu)

***********
[etau]

In [20]:
train_single_mass(xs_dict_new, xb_dict_new, 'etau', -3, '500GeV', "model_mass_500_etau_new", epochs=30, selected_features = selected_features_etau)

*********
[mutau]

In [19]:
train_single_mass(xs_dict_new, xb_dict_new, 'mutau', -2, '500GeV', "model_mass_500_mutau_new", epochs=30, selected_features = selected_features_mutau)

### B) mass = 2000 GeV

#### a) old ntuple
[emu]

In [15]:
train_single_mass(xs_dict_old, xb_dict_old, 'emu', -4, '2000GeV', "model_mass_2000_emu", epochs=30, 
                  selected_features = selected_features_emu, select_bkg_mass = True,
                  self_defined_mass = False, minm = 1800, maxm = 2200)

*********
[etau] 

In [13]:
train_single_mass(xs_dict_old, xb_dict_old, 'etau', -3, '2000GeV', "model_mass_2000_etau", epochs=30, 
                  selected_features = selected_features_etau, select_bkg_mass = True,
                  self_defined_mass = False, minm = 1800, maxm = 2200)

*********
[mutau]

In [11]:
train_single_mass(xs_dict_old, xb_dict_old, 'mutau', -2, '2000GeV', "model_mass_2000_mutau", epochs=30, selected_features = selected_features_mutau)

#### b) new ntuple
[emu] too few background

In [16]:
train_single_mass(xs_dict_new, xb_dict_new, 'emu', -4, '2000GeV', "model_mass_2000_emu_new", epochs=30, 
                  selected_features = selected_features_emu, select_bkg_mass = True,
                  self_defined_mass = False, minm = 1800, maxm = 2200)

********
[etau]

In [68]:
train_single_mass(xs_dict_new, xb_dict_new, 'etau', -3, '2000GeV', "model_mass_2000_etau_new", epochs=30, 
                  selected_features = selected_features_etau, select_bkg_mass = True, 
                  self_defined_mass = True, minm = 1800, maxm = 2200)

*******
[mutau]

In [70]:
train_single_mass(xs_dict_new, xb_dict_new, 'mutau', -2, '2000GeV', "model_mass_2000_mutau_new", epochs=30, 
                  selected_features = selected_features_mutau, select_bkg_mass = True,
                  self_defined_mass = True, minm = 1800, maxm = 2200)

## 2 - Train with 500 GeV mass point and full mass point (use old ntuple)

### A) Train with 500 GeV single mass point
#### a) emu

In [33]:
model_single = train_single_mass(xs_dict_old, xb_dict_old, 'emu', -4, '500GeV', "model_mass_500_emu_single", epochs=30, 
                                 selected_features = selected_features_emu, select_bkg_mass = True)

### B) Train with full mass point

In [42]:
model_total = train_single_mass(xs_dict_old, xb_dict_old, 'emu', -4, 'all', "model_mass_all_emu", epochs=30, 
                                 selected_features = selected_features_emu, select_bkg_mass = True)

### B1) Train with full mass point, normalized before adding all mass point samples

In [11]:
model_total_norm = train_single_mass(xs_dict_old, xb_dict_old, 'emu', -4, 'all_norm', "model_mass_all_emu", epochs=30, 
                                 selected_features = selected_features_emu, select_bkg_mass = True)

### B2) Train with max 2400 GeV

In [62]:
model_max_2400 = train_single_mass(xs_dict_old, xb_dict_old, 'emu', -4, 'max_2400', "model_mass_all_emu", epochs=30, 
                                 selected_features = selected_features_emu, select_bkg_mass = True)

### B3) Train with max 2400 GeV, normalized before adding signal samples together

In [56]:
model_max_2400_norm = train_single_mass(xs_dict_old, xb_dict_old, 'emu', -4, 'max_2400_norm', "model_max_2400_norm", epochs=30, 
                                 selected_features = selected_features_emu, select_bkg_mass = True)

### B4) train with 3-point (normalized before add)
train with 500, 1000, 2000GeV

In [12]:
model_3_point_norm = train_single_mass(xs_dict_old, xb_dict_old, 'emu', -4, '3_point_norm', "model_3_point_norm", epochs=30, 
                                       selected_features = selected_features_emu, select_bkg_mass = True, 
                                       self_defined_mass = True, minm = 450, maxm = 2200)

### B4) train with 4-point (normalized before add)
train with 500, 700, 1000, 2000GeV

In [18]:
model_4_point_norm = train_single_mass(xs_dict_old, xb_dict_old, 'emu', -4, '4_point_norm', "model_4_point_norm", epochs=30, 
                                       selected_features = selected_features_emu, select_bkg_mass = True, 
                                       self_defined_mass = True, minm = 450, maxm = 2200)

### C) Compare around 500GeV
set mass range to (450, 550)

In [66]:
mass_min = 450
mass_max = 550
xs_emu = modify_array(xs_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4,
                      select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max)
xb_emu = modify_array(xb_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4,
                      select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max)
xs_emu_selected = get_part_feature(xs_emu, selected_features_emu)
xb_emu_selected = get_part_feature(xb_emu, selected_features_emu)

# single mass train model
print "use model:", model_single.model_name
fig, ax = plt.subplots()
ax.hist(model_single.get_model().predict(xs_emu_selected), bins=50, 
        histtype='step', label='signal', density=True)
ax.hist(model_single.get_model().predict(xb_emu_selected), bins=50, 
        histtype='step', label='background', density=True)
ax.set_title('training scores')
ax.legend(loc='upper center')
ax.set_xlabel("Output score")
ax.set_ylabel("arb. unit")
ax.grid()
plt.show()

# full mass train model
print "use model:", model_total_norm.model_name
fig, ax = plt.subplots()
ax.hist(model_total_norm.get_model().predict(xs_emu_selected), bins=50, 
        histtype='step', label='signal', density=True)
ax.hist(model_total_norm.get_model().predict(xb_emu_selected), bins=50, 
        histtype='step', label='background', density=True)
ax.set_title('training scores')
ax.legend(loc='upper center')
ax.set_xlabel("Output score")
ax.set_ylabel("arb. unit")
ax.grid()
plt.show()

# below 2400 mass train model, normed
print "use model:", model_max_2400_norm.model_name
fig, ax = plt.subplots()
ax.hist(model_max_2400_norm.get_model().predict(xs_emu_selected), bins=50, 
        histtype='step', label='signal', density=True)
ax.hist(model_max_2400_norm.get_model().predict(xb_emu_selected), bins=50, 
        histtype='step', label='background', density=True)
ax.set_title('training scores')
ax.legend(loc='upper center')
ax.set_xlabel("Output score")
ax.set_ylabel("arb. unit")
ax.grid()
plt.show()

# 3 mass point train model, normed
print "use model:", model_3_point_norm.model_name
fig, ax = plt.subplots()
ax.hist(model_3_point_norm.get_model().predict(xs_emu_selected), bins=50, 
        histtype='step', label='signal', density=True)
ax.hist(model_3_point_norm.get_model().predict(xb_emu_selected), bins=50, 
        histtype='step', label='background', density=True)
ax.set_title('training scores')
ax.legend(loc='upper center')
ax.set_xlabel("Output score")
ax.set_ylabel("arb. unit")
ax.grid()

## 3 - train with new and old signal to see the result

### A) 500GeV

In [10]:
print "Training new_vs_old. ({})".format("emu")
# get data
print "Loading signal."
xs_emu_new = modify_array(xs_dict_new['500GeV'], weight_id = -1, select_channel = True, channel_id = -4,
                          norm = True, shuffle = True, shuffle_seed = int(time.time()))
xs_emu_old = modify_array(xs_dict_old['500GeV'], weight_id = -1, select_channel = True, channel_id = -4,
                          norm = True, shuffle = True, shuffle_seed = int(time.time()))
# set model and train
model_deep = model.model_0913("new_vs_old", len(selected_features_emu))
model_deep.compile()
model_deep.prepare_array(xs_emu_new, xs_emu_old, selected_features_emu)
model_deep.train(epochs = 30, verbose = 1)
# performance plots
model_deep.show_performance()
print "\nDone."

print "Training old_vs_new.".format("emu")
# set model and train
model_deep = model.model_0913("olg_vs_new", len(selected_features_emu))
model_deep.compile()
model_deep.prepare_array(xs_emu_old, xs_emu_new, selected_features_emu)
model_deep.train(epochs = 30, verbose = 1)
# performance plots
model_deep.show_performance()
print "\nDone."

### B) 2000 GeV

In [11]:
print "Training new_vs_old. ({})".format("emu")
# get data
print "Loading signal."
xs_emu_new = modify_array(xs_dict_new['2000GeV'], weight_id = -1, select_channel = True, channel_id = -4,
                          norm = True, shuffle = True, shuffle_seed = int(time.time()))
xs_emu_old = modify_array(xs_dict_old['2000GeV'], weight_id = -1, select_channel = True, channel_id = -4,
                          norm = True, shuffle = True, shuffle_seed = int(time.time()))
# set model and train
model_deep = model.model_0913("new_vs_old_2000", len(selected_features_emu))
model_deep.compile()
model_deep.prepare_array(xs_emu_new, xs_emu_old, selected_features_emu)
model_deep.train(epochs = 30, verbose = 1)
# performance plots
model_deep.show_performance()
print "\nDone."

print "Training old_vs_new.".format("emu")
# set model and train
model_deep = model.model_0913("olg_vs_new_2000", len(selected_features_emu))
model_deep.compile()
model_deep.prepare_array(xs_emu_old, xs_emu_new, selected_features_emu)
model_deep.train(epochs = 30, verbose = 1)
# performance plots
model_deep.show_performance()
print "\nDone."

## 4 Plot auc vs mass (with full mass train & 3-point mass train)
### A) Use model_total_norm (full mass train)
use mass +- 50GeV bin 

In [44]:
model_used = model_total_norm

xs_emu = modify_array(xs_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4,
                      select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max)
xb_emu = modify_array(xb_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4,
                      select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max)
xs_emu_selected = get_part_feature(xs_emu, selected_features_emu)
xb_emu_selected = get_part_feature(xb_emu, selected_features_emu)
mass_list = []
auc_list = []
for i in range(0, 16):
    mass = i * 100 + 500
    mass_min = mass - 50
    mass_max = mass + 50
    xs_emu = modify_array(xs_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4,
                          select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max)
    xb_emu = modify_array(xb_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4,
                          select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max)
    xs_emu_selected = get_part_feature(xs_emu, selected_features_emu)
    xb_emu_selected = get_part_feature(xb_emu, selected_features_emu)
    ys_emu = np.ones(len(xs_emu_selected))
    yb_emu = np.zeros(len(xb_emu_selected))
    x_emu = np.concatenate((xs_emu_selected, xb_emu_selected))
    y_emu = np.concatenate((ys_emu, yb_emu))
    predicti_y = model_used.get_model().predict(x_emu)
    fpr, tpr, threshold = roc_curve(y_emu, predicti_y)
    auc_value = auc(fpr, tpr)
    mass_list.append(mass)
    auc_list.append(auc_value)
plt.plot(mass_list, auc_list)
plt.ylim((0, 1))
plt.xlabel("mass GeV")
plt.ylabel("auc")
plt.show()

### B) Use model_3_point_norm (3-point mass train)
use mass +- 50GeV bin 

In [20]:
model_used = model_3_point_norm

mass_list = []
auc_list = []
for i in range(0, 16):
    mass = i * 100 + 500
    mass_min = mass - 50
    mass_max = mass + 50
    xs_emu = modify_array(xs_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4,
                          select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max)
    xb_emu = modify_array(xb_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4,
                          select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max)
    xs_emu_selected = get_part_feature(xs_emu, selected_features_emu)
    xb_emu_selected = get_part_feature(xb_emu, selected_features_emu)
    ys_emu = np.ones(len(xs_emu_selected))
    yb_emu = np.zeros(len(xb_emu_selected))
    x_emu = np.concatenate((xs_emu_selected, xb_emu_selected))
    y_emu = np.concatenate((ys_emu, yb_emu))
    predicti_y = model_used.get_model().predict(x_emu)
    fpr, tpr, threshold = roc_curve(y_emu, predicti_y)
    auc_value = auc(fpr, tpr)
    mass_list.append(mass)
    auc_list.append(auc_value)
plt.plot(mass_list, auc_list)
plt.ylim((0, 1))
plt.xlabel("mass GeV")
plt.ylabel("auc")
plt.show()

### B) Use model_4_point_norm (4-point mass train)
use mass +- 50GeV bin 

In [21]:
model_used = model_4_point_norm

mass_list = []
auc_list = []
for i in range(0, 16):
    mass = i * 100 + 500
    mass_min = mass - 50
    mass_max = mass + 50
    xs_emu = modify_array(xs_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4,
                          select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max)
    xb_emu = modify_array(xb_dict_old['all'], weight_id = -1, select_channel = True, channel_id = -4,
                          select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max)
    xs_emu_selected = get_part_feature(xs_emu, selected_features_emu)
    xb_emu_selected = get_part_feature(xb_emu, selected_features_emu)
    ys_emu = np.ones(len(xs_emu_selected))
    yb_emu = np.zeros(len(xb_emu_selected))
    x_emu = np.concatenate((xs_emu_selected, xb_emu_selected))
    y_emu = np.concatenate((ys_emu, yb_emu))
    predicti_y = model_used.get_model().predict(x_emu)
    fpr, tpr, threshold = roc_curve(y_emu, predicti_y)
    auc_value = auc(fpr, tpr)
    mass_list.append(mass)
    auc_list.append(auc_value)
plt.plot(mass_list, auc_list)
plt.ylim((0, 1))
plt.xlabel("mass GeV")
plt.ylabel("auc")
plt.show()

# test area

In [ ]:
test_dict = {}
a = 1
b = 2
test_dict['a'] = a
test_dict['b'] = b
print test_dict

a = 10
b = 20
print test_dict

test_dict['a'] = 100
print test_dict
print a

In [ ]:
from lfv_pdnn_code_v1.common import print_helper

print_helper.print_warning("test  warning")

In [18]:
test_list = ["a", 'b', 'c']
print 'a' in test_list

In [1]:
import datetime

print datetime.date.today().strftime("_%m_%d_%Y")

In [20]:
import os
test_path = "/mnt/asdfasfas"
print os.path.split(test_path)[0]